In [1]:
import tushare as ts
import pandas as pd
pd.set_option('display.max_columns', None)

import shutil
import os

start_year = 2009
gap_year = 9  # 9年作为训练，之后一年用来测试， 2009-2018 2019
pro = pro = ts.pro_api(
    '19c1ab37c30ca784d8658cf1050c0aba8eef6cb8d7cda5d0744d6bfb')

In [ ]:
columns_to_drop = ['ts_code', 'change', 'pct_chg', 'amount']  # 删除指定的列

if not os.path.exists('dataset'):
    os.mkdir('dataset')
else:
    shutil.rmtree('dataset')
    os.mkdir('dataset')

df = pro.daily(ts_code='000001.SZ', start_date='20090101', end_date='20240101')
df.drop(columns=columns_to_drop, inplace=True)
df['trade_date'] = pd.to_datetime(df["trade_date"], format="%Y%m%d")
df_reversed = df[::-1]
df_reversed.to_csv('./dataset/reversed_all_data.csv',
                   index=False, header=False)  # index=False 防止保存索引列

In [4]:
df = pro.index_basic(market='SSE')
df.to_csv('./index.csv')